In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
from pathlib import Path

### import jupyter notebook of Practical C in order to use and not redefine useful functions used there
import import_ipynb
from practicalMixGaussC import multivariate_gaussian_probability_vectorized
from practicalMixGaussC import multivariate_gaussian_probability
from practicalMixGaussC import log_likelihood_vectorized
from practicalMixGaussC import E_step_vectorized
from practicalMixGaussC import M_step_vectorized

In [ ]:
### path where apples images and masks are stored
train_images_path = 'apples/apples'

In [ ]:
def load_filenames(path):
    ### find all apple images and masks
    included_extensions = ['jpg','jpeg','png']
    file_names = [fn for fn in os.listdir(path) if any(fn.endswith(ext) for ext in included_extensions)]
    return file_names

In [ ]:
file_names = load_filenames(train_images_path)

In [ ]:
print(file_names)

In [ ]:
### create the whole path of each file
files = []
for f in file_names:
    path_of_file = train_images_path + '/' + f
    files.append(path_of_file)

In [ ]:
## sort the files by name
files = sorted(files)

In [ ]:
print(files)

In [ ]:
## create 
pairs = []
i = 0
while i < (len(files) - 1):
    pairs.append((files[i],files[i+1]))
    i += 2
print(pairs)

In [ ]:
def normalize_image(image):
    return image / 255

In [ ]:
def create_binary_mask(mask):

    ## Read ground the ground truth mask
    ## Create a binary matrix representation of the mask we read.
    
    height = np.array(mask).shape[0]
    width = np.array(mask).shape[1]
    
    ## the third dimension of the third dimension of the mask determines if the pixel is apple or non apple pixel 

    binary_mask = np.zeros((height,width))
    for i in range(0,height):
        for j in range(0,width):
            if(mask[i][j][2] != 0):
                binary_mask[i][j] = 1
            else:
                binary_mask[i][j] = 0
                
    return binary_mask

In [ ]:
# In these two lists we store the pixels of each class apple/no apple
apple_pixels = []
non_apple_pixels = []

## process in pairs every apple image and its ground tuth mask
for (image,ground_truth) in pairs:
    
    print("Processing pair of images: " + str((image,ground_truth)))
    
    ## read the specific image
    image = plt.imread(image)
    
    print("Shape of the image is: " + str(image.shape))
    
    ## normalise the image, diving by the max value of a pixel in RGB
    ## in this way the max value is set to 1
    image = normalize_image(image)
    
    mask = plt.imread(ground_truth)
    
    ## create a binary mask of 0 and 1 determing which pixels refer to apples and which not
    binary_mask = create_binary_mask(mask)
    
    ## create a list of pixels for each of the two classes apples/mon apples
    ## Based on the image and the mask we read
    for i in range(binary_mask.shape[0]):
        for j in range(binary_mask.shape[1]):
            ## if the respective pixel corresponds to apple class
            if binary_mask[i, j] != 0:
                apple_pixels.append(image[i,j])
            ## if the respective pixels does not correspond to apple class
            else:
                non_apple_pixels.append(image[i,j])
                
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
    ax1.imshow(image)
    ax1.set_title('Image')
    ax2.imshow(binary_mask)
    ax2.set_title('Ground Truth - Binary Mask')
    
plt.show()

In [ ]:
### Here we compute the total pixelss we have available for the training
### We calculate also two priors (Apple and Non Apple Prior)

apple_pixels = np.asarray(apple_pixels).T
non_apple_pixels = np.asarray(non_apple_pixels).T

print("Apple Pixels have shape of: " + str(apple_pixels.shape))
print("Non apple Pixels have shape of: " + str(non_apple_pixels.shape))

### total number of pixels apples + non apples
total_pixels = apple_pixels.shape[1] + non_apple_pixels.shape[1]
apple_prior = apple_pixels.shape[1] / total_pixels
non_apple_prior = non_apple_pixels.shape[1] / total_pixels

print("Total pixels: " + str(total_pixels))
print("Apple Prior: " + str(apple_prior))
print("Non Apple Prior: " + str(non_apple_prior))

In [ ]:
## calculate mean and covariance
def calculate_mean_and_cov(data):
    mean = np.mean(data, axis=1)
    return (mean,(1 / data.shape[1] * (data - mean[:, None]) @ (data - mean[:, None]).T))

In [ ]:
def model_initialization(data,k):
    
    ## number of pixels and number of channels we have 
    nDims, nData = data.shape
    
    ## calculate the mean and the covariance of the data
    mean,cov = calculate_mean_and_cov(data)
    
    init_type = 0
    
    mixGaussEst = dict()
    mixGaussEst['d'] = nDims
    mixGaussEst['k'] = k
    
    if(init_type==1):
        mixGaussEst['weight'] = (1 / k) * np.ones(shape=(k))
        mixGaussEst['mean'] = mean[:, None] * (0.5 + np.random.uniform(size=(k)))
        mixGaussEst['cov'] = \
        (2 + 0.4 * np.random.normal(size=(k)))[None, None] * cov[:, :, None] 
    else:
        mixGaussEst['weight'] = (1 / k) * np.ones(shape=(k))
        mixGaussEst['mean'] = 2 * np.random.randn(nDims, k)
        mixGaussEst['cov'] = np.zeros(shape=(nDims, nDims, k))
        for cGauss in range(k):
            mixGaussEst['cov'][:, :, cGauss] = 2.5 + 1.5 * np.random.uniform() * np.eye(nDims)
    
    return mixGaussEst

In [ ]:
def plot_likelihood(iterations,likelihood):
    
    plt.plot(iterations, likelihood, color='green', label='Log-Likelihood')
    plt.title('Log Likelihood over iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Log-Likelihood')
    plt.legend()
    plt.show()
    plt.figure()

In [ ]:
## define a criteria to deterime when to stop training
## if abs(log_likelihood(i) - log_likelihood(i-1) < 0.001
## if the is minor change on the loglikelihood through 2 consecutive iterations the stop training
## and return the current model
def stop_training_criteria(training_threshhold,log_list,index):
    if(abs(log_list[index] - log_list[index-1]) < training_threshhold):
        return True
    else:
        return False

Here we fit our Gaussian Mixture Model

In [ ]:
def GaussianMixtureModel(data,k,training_threshhold):
    
        mixGaussEst = model_initialization(data,k)
    
        ## initialize the responsibilities with zeros
        responsibilities = np.zeros(shape=(k,data.shape[1]))
        
        ## compute the log likelihood for the initial arbitrary configuration of parameters
        log_lkl = log_likelihood_vectorized(data, mixGaussEst)
        
        print("Log likelihood before the first iter is: " + str(log_lkl))


        # Maximum Number of iterations
        ## The training terminates earlier if a training stop criteria we set is satisfied
        max_iterations = 300
        
        iter_list = []
        log_list = []

        for cIter in range(max_iterations):
            
            iter_list.append(cIter)
            
            log_list.append(log_lkl)
            
            ## perform expectation step and update the responsibilities
            responsibilities = E_step_vectorized(responsibilities,mixGaussEst,data)
            
            ## perform maximisation step and update the model parameters based on the new responsibilities
            mixGaussEst = M_step_vectorized(responsibilities,mixGaussEst,data)
            
            log_lkl = log_likelihood_vectorized(data, mixGaussEst)
            
            print("Log likelihood of iter: " + str(cIter) + " is " + str(log_lkl))
            
            if(cIter > 1):
                if(stop_training_criteria(training_threshhold,log_list,cIter) == True):
                    print("Training stoped at iteration: "  + str(cIter) + " with log likelihood: " + str(log_lkl) + " as no further improvement was observed")
                    plot_likelihood(iter_list,log_list)
                    return mixGaussEst 
            
        plot_likelihood(iter_list,log_list)
            
        return mixGaussEst

In [ ]:
training_threshhold = 0.001
Apple_Gaussians = 3
Non_Apple_Gaussians = 3

apple_model = GaussianMixtureModel(apple_pixels,Apple_Gaussians,training_threshhold)
non_apple_model = GaussianMixtureModel(non_apple_pixels,Non_Apple_Gaussians,training_threshhold)

In [ ]:
def likelihood_vectorized(data, mixGaussEst):
        result = np.zeros((mixGaussEst['k'],data.shape[1]))
        for k in range(mixGaussEst['k']):
            result[k,:] = mixGaussEst['weight'][k]*multivariate_gaussian_probability_vectorized(data, mixGaussEst['mean'][:, k], mixGaussEst['cov'][:, :, k])
        return np.sum(result, axis=0)

In [ ]:
def likelihood(data, mixGaussEst):
        result = 0
        for k in range(mixGaussEst['k']):
            result += mixGaussEst['weight'][k]*multivariate_gaussian_probability(data, mixGaussEst['mean'][:, k], mixGaussEst['cov'][:, :, k])
        return result

In [ ]:
## given an image find the posterior distribution of each pixel be an apple pixel
def find_posterior(image,apple_model,non_apple_model):
    
    rows,columns,channels = image.shape
    
    reshaped_image = image.reshape(rows*columns,channels)
    
    apple_likelihood = []
    non_apple_likelihood = []
    
    for index in range(reshaped_image.shape[0]):
        pixel = reshaped_image[index].T
        apple_likelihood.append(likelihood(pixel,apple_model))
        non_apple_likelihood.append(likelihood(pixel,non_apple_model))

    apple_likelihood = np.array(apple_likelihood).reshape(rows,columns)
    non_apple_likelihood = np.array(non_apple_likelihood).reshape(rows,columns)
    
    ## compute the normalisation constant of the posterior distribution
    normalisation_constant = (apple_likelihood * apple_prior)+ (non_apple_likelihood * non_apple_prior)
        
    ## compute the apple posterior distribution of apple    
    apple_posterior = apple_likelihood * apple_prior / normalisation_constant

    return apple_posterior

In [ ]:
# maybe reform this function

def find_posterior_vectorized(image,apple_model,non_apple_model):
    
    width,height,channels = image.shape
    
    reshaped_image = image.reshape(height*width,channels)
       
    ## compute the apple likelihood of each pixel  
    apple_likelihood = likelihood_vectorized(reshaped_image.T,apple_model).reshape(width,height)
    non_apple_likelihood = likelihood_vectorized(reshaped_image.T,non_apple_model).reshape(width,height)
    
    ## compute the normalisation constant of the posterior distribution
    normalisation_constant = (apple_likelihood * apple_prior)+ (non_apple_likelihood * non_apple_prior)
        
    ## compute the apple posterior distribution of apple    
    apple_posterior = apple_likelihood * apple_prior / normalisation_constant

    return apple_posterior

In [ ]:
def ROC_Curve_Plot(false_positive_rate,true_positive_rate):
    
    plt.plot(false_positive_rate, true_positive_rate, color='red', label='ROC-Curve')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.legend()
    plt.show()
    plt.figure()    

In [ ]:
def evaluate(apple_posterior,binary_mask,threshhold_step_size):
    
    print("Evaluating POSTERIOR for threshhold step: " + str(threshhold_step_size))
    
    ## create a threshhold range
    thresh_hold_range = np.arange(0,1,threshhold_step_size)
    
    tpr_list = []
    fpr_list = []
    
    for index in range(len(thresh_hold_range)):
        current_threshhold = thresh_hold_range[index]
        
        print("Evaluating posterior for threshhold: " + str(current_threshhold))
        
        true_positive = 0
        true_negative = 0
        false_positive = 0
        false_negative = 0
        
        true_positive_rate = 0
        false_positive_rate = 0
        
        for i in range(apple_posterior.shape[0]):
            for j in range(apple_posterior.shape[1]):
                if((binary_mask[i][j] == 1) and (apple_posterior[i][j] > current_threshhold)):
                    true_positive += 1
                elif ((binary_mask[i][j] == 1) and (apple_posterior[i][j] < current_threshhold)):
                    false_positive += 1
                elif ((binary_mask[i][j] == 0) and (apple_posterior[i][j] < current_threshhold)):
                    true_negative += 1
                elif ((binary_mask[i][j] == 0) and (apple_posterior[i][j] > current_threshhold)):
                    false_negative += 1
                    
        print("Evaluating for threshhold: " + str(current_threshhold) + " tp-tn-fp-fn:  " + str(true_positive) + "-" + str(true_negative) + "-" + str(false_positive) + "-" + str(false_negative))
          
        denom_1 = true_positive + false_negative
        denom_2 = false_positive + true_negative
        
        if(denom_1 == 0):
            true_positive_rate = 0
        else:
            true_positive_rate = true_positive/(denom_1)
            
        if(denom_2 == 0):
            false_positive_rate = 0
        else:
            false_positive_rate = false_positive/(denom_2)
        
        tpr_list.append(true_positive_rate)
        fpr_list.append(false_positive_rate)
        
    ROC_Curve_Plot(fpr_list,tpr_list)

In [ ]:
threshhold_step_size = 0.2
num_of_testing_images = 3
testing_directory = 'testApples'

for i in range(0,num_of_testing_images):
    
    image_name = testing_directory + '/' + 'apple_' + str(i+1) + '.jpg'
    mask_name = testing_directory + '/' + 'apple_' + str(i+1) + '_mask.png'
    
    print("Testing image: " + image_name)
    
    maskf = Path(mask_name)
    
    mask_flag = 0
    if maskf.is_file():
        print("A Mask for this file exists and is: " + mask_name)
        mask_flag = 1
    else:
        print("A Mask for this file does not exist!")
      
    ## read the image and normalise it
    img = plt.imread(image_name)
    ## normalize image
    img = normalize_image(img)
    
    ## create the posterior mask for this image
    apple_posterior = find_posterior(img,apple_model,non_apple_model)
    
    ## create the posterior mask for this image
    #apple_posterior = find_posterior_vectorized(img,apple_model,non_apple_model)
    
    ## if a ground truth mask exists for that image
    ## evaluate the results using ROC_CURVE
    if(mask_flag == 1):
        ## read the respective mask file
        mask = plt.imread(maskf)
        ## create a binary mask of 0 and 1 determing which pixels refer to apples and which not
        binary_mask = create_binary_mask(mask)
        evaluate(apple_posterior,binary_mask,threshhold_step_size)
  
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 7))
    ax1.imshow(img)
    ax1.set_title('Image')
    ax2.imshow(apple_posterior)
    ax2.set_title('Posterior')
    plt.show()